In [1]:
setwd(paste0(Sys.getenv('ROOT'), '/R'))
getwd()

[1] "/mnt/hdd/develop/kaggle/house-prices/R/R"

In [8]:
source('main.R')

data <- kaggle.house$loadData()

df.combined <- kaggle.house$getCombinedDataset(data$train, data$test) %>% (kaggle.house$na$fixAll)

dataset <- 
    df.combined %>% 
    filter(dataSource == "train") %>% 
    mutate(sale_price_log = log(SalePrice)) %>%
    select(OverallQual, Neighborhood, SalePrice)

In [9]:
dataset %>%
    group_by(Neighborhood, OverallQual) %>%
    count

Neighborhood,OverallQual,n
Blmngtn,7,14
Blmngtn,8,3
Blueste,6,2
BrDale,5,5
BrDale,6,11
BrkSide,1,1
BrkSide,2,1
BrkSide,3,3
BrkSide,4,10
BrkSide,5,21
